In [1]:
import pandas as pd

file_path = 'hks_scores.xlsx'
excel_data = pd.ExcelFile(file_path)

all_players = pd.DataFrame()

for sheet_name in excel_data.sheet_names:
    sheet_df = excel_data.parse(sheet_name, header=None)

    quiz_name = sheet_df.iloc[0, 0]
    max_points = sheet_df.iloc[0, 1]

    # Extract player data.
    players_data = sheet_df.iloc[1:].copy()
    players_data.columns = ['Player', 'Score']
    players_data['Player'] = players_data['Player'].replace('Krešimir Sučević Međeral', 'Krešimir Sučević-Međeral')
    
    # Calculate the scaled score based on the top score.
    top_score = players_data['Score'].max()
    players_data['Scaled Score'] = (players_data['Score'] / top_score) * max_points
    players_data['Quiz'] = quiz_name
    players_data['Max Points'] = max_points

    all_players = pd.concat([all_players, players_data], ignore_index=True)

# Pivot the data to have one row per player and one column per quiz.
all_players_pivot = all_players.pivot_table(
    index='Player', 
    columns='Quiz', 
    values='Scaled Score', 
    aggfunc='first'
).fillna(0)

all_players_pivot.columns.name = None
all_players_pivot = all_players_pivot.reset_index()

hr_12x7_quizzes = [quiz for quiz in all_players_pivot.columns if ("12x7" in quiz or "Hrvatskih 100" in quiz)]
other_quizzes = [quiz for quiz in all_players_pivot.columns if quiz not in hr_12x7_quizzes + ["Player"]]

def calculate_total(row):
    hr_12x7_quizzes_scores = sorted([row[quiz] for quiz in hr_12x7_quizzes], reverse=True)
    other_scores = sorted([row[quiz] for quiz in other_quizzes], reverse=True)
    total_score = sum(hr_12x7_quizzes_scores[:-2]) + sum(other_scores[:-2])
    return total_score

all_players_pivot['Total Score'] = all_players_pivot.apply(calculate_total, axis=1)
all_players_pivot = all_players_pivot.sort_values(by='Total Score', ascending=False)

all_players_pivot = all_players_pivot.reset_index(drop=True)
all_players_pivot.index = range(1, len(all_players_pivot) + 1)

columns = ['Player', 'Total Score'] + [col for col in all_players_pivot.columns if col not in ['Player', 'Total Score']]
all_players_pivot = all_players_pivot[columns]

In [2]:
all_players_pivot.to_excel('test.xlsx')

In [101]:
all_players_pivot[all_players_pivot['Player'] == 'Lovro Jurišić']

,Player,Total Score,12x7-5x2,12x7-5x3,12x7-5x4,12x7-5x5,12x7-5x6,12x7-5x7,12x7-6x1,Austrian Open 2024,Cro Slam 2024-02,Cro Slam 2024-06,Cro Slam 2024-10,Croatian Open 2024,MMM 2024,Norway Open 2024,Serbian Open 2024,WQC 2024
2,Lovro Jurišić,1487.258923,80.0,70.30303,80.0,78.461538,71.724138,61.111111,76.307692,150.0,138.129496,142.758621,147.87234,190.643275,147.413793,138.851351,140.517241,181.560284


In [102]:
all_players_pivot.to_excel('test.xlsx')

In [3]:
import pandas as pd

class QuizRankingGenerator:
    def __init__(self, file_path):
        self.file_path = file_path
        self.excel_data = pd.ExcelFile(file_path)
        self.all_players = pd.DataFrame()
        self.pivoted_data = None
        self.quiz_order = []  # To preserve quiz order
        self.hr_12x7_quizzes = []
        self.other_quizzes = []

    def process_sheet(self, sheet_name):
        sheet_df = self.excel_data.parse(sheet_name, header=None)
        quiz_name = sheet_df.iloc[0, 0]
        max_points = sheet_df.iloc[0, 1]

        # Save quiz order
        if quiz_name not in self.quiz_order:
            self.quiz_order.append(quiz_name)

        # Extract player data.
        players_data = sheet_df.iloc[1:].copy()
        players_data.columns = ['Player', 'Score']
        players_data['Player'] = players_data['Player'].replace('Krešimir Sučević Međeral', 'Krešimir Sučević-Međeral')
        
        # Calculate scaled scores.
        top_score = players_data['Score'].max()
        players_data['Scaled Score'] = (players_data['Score'] / top_score) * max_points
        players_data['Quiz'] = quiz_name
        players_data['Max Points'] = max_points

        self.all_players = pd.concat([self.all_players, players_data], ignore_index=True)

    def process_all_sheets(self):
        for sheet_name in self.excel_data.sheet_names:
            self.process_sheet(sheet_name)

    def pivot_scores(self):
        self.pivoted_data = self.all_players.pivot_table(
            index='Player',
            columns='Quiz',
            values='Scaled Score',
            aggfunc='first'
        ).fillna(0)
        self.pivoted_data.columns.name = None
        self.pivoted_data = self.pivoted_data.reset_index()

        # Reorder columns based on quiz_order
        self.pivoted_data = self.pivoted_data[['Player'] + self.quiz_order]

    def categorize_quizzes(self):
        self.hr_12x7_quizzes = [quiz for quiz in self.quiz_order if "12x7" in quiz]
        self.other_quizzes = [quiz for quiz in self.quiz_order if quiz not in self.hr_12x7_quizzes]

    def calculate_total_scores(self):
        def calculate_partial_scores(row):
            hr_12x7_scores = sorted([row[quiz] for quiz in self.hr_12x7_quizzes], reverse=True)
            other_scores = sorted([row[quiz] for quiz in self.other_quizzes], reverse=True)
            best_12x7 = sum(hr_12x7_scores[:-2]) if len(hr_12x7_scores) > 2 else sum(hr_12x7_scores)
            best_other = sum(other_scores[:-2]) if len(other_scores) > 2 else sum(other_scores)
            total_score = best_12x7 + best_other
            return best_12x7, best_other, total_score

        partial_scores = self.pivoted_data.apply(calculate_partial_scores, axis=1)
        self.pivoted_data['12x7 - Best 5'] = partial_scores.map(lambda x: x[0])
        self.pivoted_data['Other - Best 6'] = partial_scores.map(lambda x: x[1])
        self.pivoted_data['Total Score'] = partial_scores.map(lambda x: x[2])

    def sort_and_reformat(self):
        self.pivoted_data = self.pivoted_data.sort_values(by='Total Score', ascending=False)
        self.pivoted_data = self.pivoted_data.reset_index(drop=True)
        self.pivoted_data.index = range(1, len(self.pivoted_data) + 1)

        # Extract columns
        fixed_columns = ['Player', 'Total Score', 'Other - Best 7']
        quiz_columns = [col for col in self.quiz_order if col not in fixed_columns]
        twelve_x7_columns = [col for col in quiz_columns if "12x7" in col]
        non_twelve_x7_columns = [col for col in quiz_columns if "12x7" not in col]

        # Determine position to insert '12x7 - Best 5'
        first_twelve_x7_index = quiz_columns.index(twelve_x7_columns[0]) if twelve_x7_columns else len(quiz_columns)
        reordered_columns = (
            fixed_columns +
            non_twelve_x7_columns[:first_twelve_x7_index] +
            ['12x7 - Best 5'] +
            non_twelve_x7_columns[first_twelve_x7_index:] +
            twelve_x7_columns
        )

        # Reorder columns
        self.pivoted_data = self.pivoted_data[reordered_columns]

    def run(self):
        self.process_all_sheets()
        self.pivot_scores()
        self.categorize_quizzes()
        self.calculate_total_scores()
        self.sort_and_reformat()
        return self.pivoted_data

In [5]:
file_path = 'hks_scores.xlsx'
ranking = QuizRankingGenerator(file_path)
final_scores = ranking.run()

In [7]:
final_scores.to_excel('test.xlsx')

In [6]:
final_scores.columns

Index(['Player', 'Total Score', 'Other - Best 7', 'Serbian Open 2024',
       'Norway Open 2024', 'WQC 2024', 'Cro Slam 2024-06', 'MMM 2024',
       'Cro Slam 2024-10', 'Croatian Open 2024', 'Austrian Open 2024',
       '12x7 - Best 5', 'Hrvatskih 100', '12x7-5x2', '12x7-5x3', '12x7-5x4',
       '12x7-5x5', '12x7-5x6', '12x7-5x7', '12x7-6x1'],
      dtype='object')